In [1]:
#[*schema['fields'].keys()]


In [2]:
import pymarc

In [3]:
import gzip
from pathlib import Path
import json
import pyarrow as pa
from pyarrow import json as pajson
from pyarrow import compute as pc
from pyarrow import parquet
import polars as pl
import pymarc

schema = json.load(open("marc-schema.json"))
[*schema.keys()]

['$schema', 'title', 'description', 'url', 'fields']

In [4]:
def parse_sub(val, field_name):
    sub = defaultdict(list)
    for d in val['subfields']:
        if len(d) > 1:
            raise IndexError("WHOA, MULTIPLE FIELDS")
        subname, subval = [*d.items()][0]
        if field_name not in schema['fields'] or subname not in schema['fields'][field_name]["subfields"] or \
           schema['fields'][field_name]["subfields"][subname]['repeatable']:
            sub[subname].append(subval)
        else:
            sub[subname] = subval
    for ind in 'ind1', 'ind2':
        try:
            sub[ind] = val[ind]
        except KeyError:
            pass
    return sub

In [1]:
def parse_json(record):
    d = record
    r = defaultdict(list)
    r['leader'] = d['leader']
    for field_pair in d['fields']:
        if len(field_pair) > 1:
            print(field_pair)
            raise IndexError("HUH")
        field_name, val = [*field_pair.items()][0]  
        if isinstance(val, str):
            r[field_name] = val
            continue
        if not field_name in schema['fields'] or schema['fields'][field_name]['repeatable']:
            r[field_name].append(parse_sub(val, field_name))
        else:
            r[field_name] = parse_sub(val, field_name)

In [5]:
from collections import defaultdict
for path_in in Path("2020445551_2019").glob("*.utf8.gz"):
    print(path_in)
    if 'combined' in str(path_in):
        continue
    dest = Path("parquet/", path_in.stem).with_suffix(".parquet")
    if dest.exists():
        continue
    f = gzip.open(path_in)
    reader = pymarc.MARCReader(f,  to_unicode=True)
    with open("/tmp/f.json", "w") as fout:
        for i, record in enumerate(reader):
            d = record.as_dict()
            r = parse_dict_marc(d)

            fout.write(json.dumps(r) + "\n")
            if i % 100 == 0:
                print(i, end="\r")
            if i > 260_000:
                raise IndexError("TOO HIGH " + path_in)
            
    del reader
    as_feather = pajson.read_json("/tmp/f.json")
    parquet.write_table(as_feather, dest,
                        compression = "zstd", compression_level = 10)
    f.close()
            #key
#val

2020445551_2019/Books.All.2019.part10.utf8.gz
2020445551_2019/Books.All.2019.part19.utf8.gz
2020445551_2019/Books.All.2019.part09.utf8.gz
2020445551_2019/Books.All.2019.part26.utf8.gz
2020445551_2019/Books.All.2019.part36.utf8.gz
2020445551_2019/Books.All.2019.part44.utf8.gz
2020445551_2019/Books.All.2019.part08.utf8.gz
2020445551_2019/Books.All.2019.part18.utf8.gz
2020445551_2019/Books.All.2019.part37.utf8.gz
2020445551_2019/Books.All.2019.part27.utf8.gz
2020445551_2019/Books.All.2019.part45.utf8.gz
2020445551_2019/Books.All.2019.part11.utf8.gz
2020445551_2019/Books.All.2019.part01.utf8.gz
2020445551_2019/Books.All.2019.part25.utf8.gz
2020445551_2019/Books.All.2019.part35.utf8.gz
2020445551_2019/Books.All.2019.part03.utf8.gz
2020445551_2019/Books.All.2019.part13.utf8.gz
2020445551_2019/Books.All.2019.part12.utf8.gz
2020445551_2019/Books.All.2019.part02.utf8.gz
2020445551_2019/Books.All.2019.part46.utf8.gz
2020445551_2019/Books.All.2019.part34.utf8.gz
2020445551_2019/Books.All.2019.par

In [ ]:
schema['fields'][field_name]["subfields"][val]

In [ ]:

fs = record.fields

In [ ]:
fs[0].tag